Author: Eric He

In [1]:
import numpy as np
import pandas as pd
import requests
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup 
from lxml import etree
import json
import csv,codecs
import time

### I. Retrieving id

Create a dataframe with column name 'title', 'id','link'

In [2]:
def getAid(url):   
    ret = []
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    } 
    try:
        response = requests.get(url,headers=headers)

        #将一段文档传入BeautifulSoup的构造方法,就能得到一个文档的对象, 可以传入一段字符串
        soup = BeautifulSoup(response.text,'lxml')
    except Exception as e:
        print(e)
    
    #Storing 
    titles = []
    ids = []
    links = []
    items = soup.find(class_ = "rank-list").findAll("li")
    for x in items:
        vid_title = x.find(class_ = "info").a.getText()
        vid_link = x.find("a")["href"][2:]
        vid_id = x["data-id"]
        titles.append(vid_title)
        ids.append(vid_id)
        links.append(vid_link)
    l = list(zip(titles,ids,links))
    df = pd.DataFrame(l,columns = ['title','id','link'])
    return df

### Extract Comments

In [3]:
def text_clean(s):
    if "回复" in s:
        k = s.index(":")
        s = s[k+1:]
    if s[0] == '"':
        s = s[1:]
    if s[len(s)-1] == '"':
        s = s[:-1]
    s = re.sub("\n","",s)
    return s

In [4]:
def makeUrls(pg,aid):
    ret = []
    for i in range(pg):
        u = "https://api.bilibili.com/x/v2/reply/main?jsonp=jsonp&next="+str(i)+"&type=1&oid="+aid+"&mode=3&plat=1&_=1627952418487"
        ret.append(u)
    return ret
# l = makeUrls(3,'291880951')

In [5]:
import re
def getComments(df, num_vids):
    f = codecs.open('bilibili_Comments.csv','w','utf_8_sig')
    word_writer = csv.writer(f)
    word_writer.writerow(["comment","video_title","video_id","video_link"])
    for i in range(num_vids):
        aid = df.iloc[i].id
        title = df.iloc[i].title
        link = df.iloc[i].link
        urls = makeUrls(1,aid) #add more pages if want more comments 1page = 80ish comments
        for url in urls:
            time.sleep(2)
            headers = { 
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
            } 
            try:
                response = requests.get(url,headers=headers)
                soup = BeautifulSoup(response.text,'lxml')
            except Exception as e:
                print(e)
            t = soup.getText()
            t = t.split('"message":')
            t = t[2:len(t)]
            for comment in t:      
                comment = comment.split(',"plat"')[0]
                comment  = text_clean(comment)
                word_writer.writerow([comment,title, aid, link])

## Bullet Comments

<html><body><p>{"code":0,"message":"0","ttl":1,"data":[{"cid":380318626,"page":1,"from":"vupload","part":"大碗牢饭_BiliBili","duration":111,"vid":"","weblink":"","dimension":{"width":1280,"height":720,"rotate":0}}]}</p></body></html>

In [6]:
def get_dm(df, num_vids):
    f = codecs.open('bilibili_DanMu.csv','w','utf_8_sig')
    word_writer = csv.writer(f)
    word_writer.writerow(["dm","freq","video_title","video_id","video_link"])
    for i in range(num_vids):
        dic = {}
        time.sleep(2)
        aid = df.iloc[i].id
        link = df.iloc[i].link
        cid = conv(link)
        title = df.iloc[i].title 
        add_dm(cid, dic)
        sorted_d = sort_dic(dic)
        counter = 0
        for key in sorted_d:
#         print(key,sorted_d[key])
            counter += 1
            word_writer.writerow([key,sorted_d[key],title,aid,link])
        print("video: ", title ,"finished. Added: "+ str(counter) +" unique DanMu")

In [7]:
def conv(link):
    BV =link.split("/")[2]
    url = "https://api.bilibili.com/x/player/pagelist?bvid="+ BV + "&jsonp=jsonp"
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    } 
    try:
        response = requests.get(url,headers=headers)
    except Exception as e:
        print(e)
    dirt=json.loads(response.text)
    cid=dirt['data'][0]['cid']
    return cid

In [8]:
def add_dm(cid, dic):
    url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
    } 
    try:
        response = requests.get(url,headers=headers)
        response.encoding ="utf-8"
        soup = BeautifulSoup(response.text,'lxml')
        dms = soup.findAll("d")
    except Exception as e:
        print(e)
    # data=parse(response.text)
    for dm in dms:
        dm = dm.getText()
        if (dm in dic):
            dic[dm] = dic[dm]+1
        else:
            dic[dm] = 1

In [9]:
def sort_dic(dic):
    sorted_dict = {}
    sorted_keys = sorted(dic, key=dic.get, reverse=True)  # [1, 3, 2]
    for w in sorted_keys:
        sorted_dict[w] = dic[w]
    return sorted_dict


In [1]:
def crawl():
    inp = input('Enter number of videos(<50)：')
    inp = int(inp)
    rank_pop = "https://www.bilibili.com/v/popular/rank/all"
    df = getAid(rank_pop)
    get_dm(df,inp)
    print("Finished dm!")
    getComments(df,inp)
    print("Finished comments!")

## Time to try

In [ ]:
crawl()